In [1]:
import pandas as pd
import numpy as np
import cv2
import glob
import tqdm
import os

In [2]:
known_plates = ['P015076', 'P015077', 'P015080', 'P015081', 'P015082', 'P015083',
                'P015084', 'P015085', 'P015090', 'P015091', 'P015092', 'P015093',
                'P015094', 'P015095', 'P015096', 'P015097', 'P015098', 'P015099']

In [3]:
fcsv_path = '/scratch-shared/phil/SPECS_phil/exp_stats/dmso_main.csv'
df = pd.read_csv(fcsv_path)

In [4]:
df = df[df.plate.isin(known_plates)]

In [5]:
df.plate.value_counts()

P015080    198
P015093    198
P015090    198
P015092    198
P015082    198
P015097    198
P015098    198
P015094    198
P015095    198
P015076    198
P015091    198
P015099    198
P015083    198
P015084    198
P015096    198
P015085    198
P015077    198
P015081    198
Name: plate, dtype: int64

In [6]:
fold = glob.glob('/mnt/micro-images-pvc/jonne' + '/specs935*')
folds = []
for i in range(len(known_plates)):
    f = [x for x in fold if known_plates[i] in x][-1]
    folds.append(f)

In [7]:
folds

['/mnt/micro-images-pvc/jonne/specs935-v1-FA-P015076-U2OS-48h-P1-L1-TL',
 '/mnt/micro-images-pvc/jonne/specs935-v1-FA-P015077-U2OS-48h-P1-L2-TL',
 '/mnt/micro-images-pvc/jonne/specs935-v1-FA-P015080-U2OS-48h-P3-L1-TL-run2',
 '/mnt/micro-images-pvc/jonne/specs935-v1-FA-P015081-U2OS-48h-P3-L2-TL',
 '/mnt/micro-images-pvc/jonne/specs935-v1-FA-P015082-U2OS-48h-P4-L1-TL',
 '/mnt/micro-images-pvc/jonne/specs935-v1-FA-P015083-U2OS-48h-P4-L2-TL',
 '/mnt/micro-images-pvc/jonne/specs935-v1-FA-P015084-U2OS-48h-P5-L1-TL',
 '/mnt/micro-images-pvc/jonne/specs935-v1-FA-P015085-U2OS-48h-P5-L2-TL',
 '/mnt/micro-images-pvc/jonne/specs935-v1-FA-P015090-U2OS-48h-P8-L1-TL',
 '/mnt/micro-images-pvc/jonne/specs935-v1-FA-P015091-U2OS-48h-P8-L2-TL',
 '/mnt/micro-images-pvc/jonne/specs935-v1-FA-P015092-U2OS-48h-P9-L1-TL-run2',
 '/mnt/micro-images-pvc/jonne/specs935-v1-FA-P015093-U2OS-48h-P9-L2-TL-run2',
 '/mnt/micro-images-pvc/jonne/specs935-v1-FA-P015094-U2OS-48h-P10-L1-TL-run2',
 '/mnt/micro-images-pvc/jonne/

In [8]:
columns = ['plate', 'well', 'path', 'nuclei', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6']
bf_df = pd.DataFrame(columns=columns)

for file_idx in range(len(known_plates)):
    files = glob.glob(sorted(glob.glob(folds[file_idx] + '/*/*'))[-1] + '/*.tif')
    files = sorted([x for x in files if '_thumb' not in x])
    
    if '-run2' in folds[file_idx] and os.path.exists(folds[file_idx].replace('-run2','')):
        files2 = glob.glob(sorted(glob.glob(folds[file_idx].replace('-run2','') + '/*/*'))[-1] + '/*.tif')
        files2 = sorted([x for x in files2 if '_thumb' not in x])
        
        files = np.concatenate((files,files2))
        files = sorted(files)
        
    d = df[df.plate == known_plates[file_idx]]
    
    for well in tqdm.tqdm(sorted(d.well.unique())):
        image_files = [x for x in files if '_' + well + '_' in x]
        for site in ['s1','s2', 's3', 's4', 's5']:
            row_files = sorted([x for x in image_files if '_' + site + '_' in x])
            
            path = '/' + np.unique([os.path.join(*x.split('/')[:-1]) for x in row_files])[0]
            
            row = {'plate':known_plates[file_idx], 'well':well, 
                   'path':path, 'nuclei':[x for x in row_files if '_w1' in x and path in x][0].split('/')[-1], 'C1':[x for x in row_files if '_w2' in x and path in x][0].split('/')[-1], 
                   'C2':[x for x in row_files if '_w3' in x and path in x][0].split('/')[-1], 'C3':[x for x in row_files if '_w4' in x and path in x][0].split('/')[-1], 'C4':[x for x in row_files if '_w5' in x and path in x][0].split('/')[-1], 
                   'C5':[x for x in row_files if '_w6' in x and path in x][0].split('/')[-1], 'C6':[x for x in row_files if '_w7' in x and path in x][0].split('/')[-1]}
            
            bf_df = bf_df.append(row, ignore_index = True)

100%|██████████| 22/22 [00:00<00:00, 39.11it/s]


In [9]:
bf_df.to_csv('/scratch-shared/phil/SPECS_phil/exp_stats/bf_dmso_main.csv', index=False)

In [10]:
#csv_path = '/scratch-shared/phil/SPECS_phil/exp_stats/bf_main.csv'
#dmso_csv_path = '/scratch-shared/phil/SPECS_phil/exp_stats/bf_dmso_main.csv'
#df = pd.read_csv(csv_path)

In [11]:
#df[df.moa.isin(moas)].moa.value_counts()